In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sb

In [3]:
import pandas as pd
from IPython.display import display

In [4]:
data = pd.read_csv('cancel1.csv')

In [5]:
pd.options.display.max_columns = None
#display(data)

In [6]:
del data['mon'],data['salesrepid'],data['due_date'],data['BUNDLE'],data['order_create_date'],\
data['canceldate'],data['FIRSTOFFEREDDATE'],data['notruckrollrequiredreason']

In [7]:
data.strIsWinbackIndicator.replace(np.nan, 'NO' ,inplace=True)

In [8]:
data.hfws_ind.replace(np.nan, 'N' ,inplace=True)

In [9]:
data.ONTselfinstallcapable.replace(np.nan, 'N' ,inplace=True)

In [10]:
data.cancel_ind.replace(np.nan, 0 ,inplace=True)

In [11]:
data.waitingdayofcustomers.replace('#VALUE!', np.nan ,inplace=True)

In [12]:
data.waitingdayforcompany.replace('#VALUE!', np.nan ,inplace=True)

In [13]:
data.shape

(52156, 23)

In [14]:
data.isnull().values.sum()

31892

In [15]:
data.dropna(axis=0, inplace = True)

In [16]:
data.groupby('cancel_ind').count()

,state,originalagency,speed,wirecenter,InstallType,competitorname,ContractType,endclientsystem,hfws_ind,waitingdayofcustomers,waitingdayforcompany,BundleType,ONTRequired,ONTInstall,strOrderType,strIsWinbackIndicator,droptype,premisetype,ONTselfinstallcapable,saleschannel,bundlename,smartcart
cancel_ind,,,,,,,,,,,,,,,,,,,,,,
0.0,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372,41372
1.0,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522,5522


In [331]:
data.head(5)

,state,originalagency,speed,wirecenter,InstallType,competitorname,ContractType,endclientsystem,hfws_ind,waitingdayofcustomers,waitingdayforcompany,BundleType,ONTRequired,ONTInstall,strOrderType,strIsWinbackIndicator,droptype,premisetype,ONTselfinstallcapable,saleschannel,bundlename,smartcart,cancel_ind
0,NY,REDVENT: REDVENT,FTTP_100M_100M,BREWSTER,TECH INSTALL,COMCAST,MONTHTOMONTH,COFEEANYWHERE,N,2,0,D,N,Y,N,NO,AERIAL,SFU,N,RED VENTURES,UNKNOWN,0,0.0
1,MD,CSSC: CSSC,FTTP_1000M_1000M,OWEN BROWN,TECH INSTALL,COMCAST,CONTRACT,COA2GO,N,3,3,TDV,N,Y,N,NO,BURIED,SFU,N,BIG BOX,1000M + TV Mundo + FDV Unlim,0,0.0
2,PA,ONLINE: ONLINE,FTTP_100M_100M,BRADDOCK,TECH INSTALL,COMCAST,MONTHTOMONTH,VZCOM,N,5,0,D,N,Y,N,NO,AERIAL,SFU,N,ONLINE CON,UNKNOWN,1,0.0
3,VA,CSSC: CSSC,FTTP_1000M_1000M,DRUMMONDS CORNER,TECH INSTALL,COX COMM,CONTRACT,COFEEANYWHERE,N,6,1,DV,Y,N,M,NO,BURIED,SFU,N,CSSC,1000M + Extreme HD - 2014 + NO FDV,0,0.0
4,NY,ONLINE: ONLINE,FTTP_1000M_1000M,LEVITTOWN,TECH INSTALL,CABLEVISION,CONTRACT,VZCOM,N,10,2,TDV,N,Y,N,NO,AERIAL,SFU,N,ONLINE CON,1000M + Preferred HD 2015 + FDV Unlim,1,0.0


In [17]:

from sklearn import preprocessing
le = preprocessing.LabelEncoder()


In [18]:
for x in data.columns:
    if (x!="waitingdayofcustomers") & (x!= "waitingdayforcompany"):
        le.fit(data[x])
        data[x]=le.transform(data[x])
        data[x] = data[x].astype('category')
    else:
        data[x]=data[x].astype('float')

#data["waitingdayofcustomers"]=pd.to_numeric(data["waitingdayofcustomers"])
#data["waitingdayofcustomers"]=data["waitingdayofcustomers"].astype(str)

In [335]:
data.dtypes


state                    category
originalagency           category
speed                    category
wirecenter               category
InstallType              category
competitorname           category
ContractType             category
endclientsystem          category
hfws_ind                 category
waitingdayofcustomers     float64
waitingdayforcompany      float64
BundleType               category
ONTRequired              category
ONTInstall               category
strOrderType             category
strIsWinbackIndicator    category
droptype                 category
premisetype              category
ONTselfinstallcapable    category
saleschannel             category
bundlename               category
smartcart                category
cancel_ind               category
dtype: object

# Feature Selection With Boruta

In [19]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [20]:
y=data['cancel_ind']
del data['cancel_ind']

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(data,y,test_size=0.2, stratify=y)

In [23]:
y = y.ravel()

In [24]:
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features - 5 features should be selected
feat_selector.fit(X_train.as_matrix(),y_train.as_matrix())
# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(X_train.as_matrix())

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	22
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	22
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	22
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	22
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	22
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	22
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	22
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	18
Tentative: 	1
Rejected: 	3


/Users/muratyalcin/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 100
Confirmed: 	18
Tentative: 	1
Rejected: 	3


/Users/muratyalcin/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	10 / 100
Confirmed: 	18
Tentative: 	1
Rejected: 	3


...

/Users/muratyalcin/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	14 / 100
Confirmed: 	18
Tentative: 	1
Rejected: 	3


/Users/muratyalcin/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	15 / 100
Confirmed: 	18
Tentative: 	1
Rejected: 	3
Iteration: 	16 / 100
Confirmed: 	18
Tentative: 	0
Rejected: 	4


BorutaPy finished running.

Iteration: 	17 / 100
Confirmed: 	18
Tentative: 	0
Rejected: 	4


/Users/muratyalcin/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


In [25]:
print (feat_selector.n_features_)

18


In [26]:
feature_df = pd.DataFrame(X_train.columns.tolist(), columns=['features'])
feature_df ['rank']=feat_selector.ranking_

In [27]:
print (feature_df.head(feat_selector.n_features_))

                 features  rank
0                   state     4
1          originalagency     1
2                   speed     1
3              wirecenter     2
4             InstallType     1
5          competitorname     1
6            ContractType     1
7         endclientsystem     1
8                hfws_ind     1
9   waitingdayofcustomers     1
10   waitingdayforcompany     1
11             BundleType     1
12            ONTRequired     1
13             ONTInstall     1
14           strOrderType     1
15  strIsWinbackIndicator     1
16               droptype     3
17            premisetype     5


# Feature Selection With SelectKBest

In [28]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2

In [29]:
selector = SelectKBest(f_classif, k = 5)
#New dataframe with the selected features for later use in the classifier. fit() method works too, if you want only the feature names and their corresponding scores
X_new = selector.fit_transform(X_train, y_train)
names = X_train.columns.values[selector.get_support()]
scores = selector.scores_[selector.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns=['Feat_names', 'F_Scores'])
#Sort the dataframe for better visualization
ns_df_sorted = ns_df.sort_values(['F_Scores', 'Feat_names'], ascending = [False, True])
print(ns_df_sorted)

              Feat_names     F_Scores
0        endclientsystem  1671.697409
4              smartcart   257.187614
1               hfws_ind   209.883373
2             BundleType   202.575290
3  strIsWinbackIndicator   162.458030


#  Feature Selection with LinearSVC

In [30]:
from sklearn import feature_selection
from sklearn.svm import LinearSVC
from sklearn.pipeline import FeatureUnion, Pipeline

# classifier
LinearSVC1 = LinearSVC(tol=1e-4,  C = 0.10000000000000001)
f5 = feature_selection.RFE(estimator=LinearSVC1, n_features_to_select=5, step=1)

pipeline = Pipeline([
    ('rfe_feature_selection', f5),
    ('clf', LinearSVC1)
    ])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('rfe_feature_selection', RFE(estimator=LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
  n_features_to_select=5, step=1, ver...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [31]:
support = pipeline.named_steps['rfe_feature_selection'].support_
feature_names = np.array(X_train.columns) # transformed list to array

feature_names[support]

array(['hfws_ind', 'ONTRequired', 'strOrderType', 'strIsWinbackIndicator',
       'smartcart'], dtype=object)

#  Feature Selection with RFE

In [34]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier

In [35]:
model = ExtraTreesClassifier()
rfe = RFE(model, 5)
fit = rfe.fit(X_train, y_train)
print("Num Features: %d" % fit.n_features_ )
print("Selected Features: %s" % fit.support_ )
print("Feature Ranking: %s" % fit.ranking_ )

Num Features: 5
Selected Features: [False False False  True False False False  True False  True  True False
 False False False False False False False  True False False]
Feature Ranking: [ 4  3  8  1 18  6 14  1 15  1  1  7 13 17 10 16  5  9 12  1  2 11]


In [36]:
feature_names = np.array(X_train.columns) # transformed list to array
feature_names[fit.support_]

array(['wirecenter', 'endclientsystem', 'waitingdayofcustomers',
       'waitingdayforcompany', 'saleschannel'], dtype=object)

## DECISION TREE

In [339]:
from sklearn.ensemble import RandomForestClassifier

In [340]:
from sklearn.tree import DecisionTreeClassifier


In [341]:
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()

In [342]:
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [343]:
y_predi = dt.predict(X_test)

In [344]:
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [345]:
nn_accy = round(accuracy_score(y_predi, y_test), 3)
print (nn_accy,"\n")
nnconfmat = confusion_matrix(y_predi,y_test)
print(nnconfmat,"\n")
nnrep = classification_report(y_predi, y_test)
print(nnrep)

0.899 

[[7758  429]
 [ 517  675]] 

             precision    recall  f1-score   support

          0       0.94      0.95      0.94      8187
          1       0.61      0.57      0.59      1192

avg / total       0.90      0.90      0.90      9379



## RANDOM FOREST

In [346]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [347]:
y_predi = rf.predict(X_test)

In [348]:
nn_accy = round(accuracy_score(y_predi, y_test), 3)
print (nn_accy,"\n")
nnconfmat = confusion_matrix(y_predi,y_test)
print(nnconfmat,"\n")
nnrep = classification_report(y_predi, y_test)
print(nnrep)

0.927 

[[8198  609]
 [  77  495]] 

             precision    recall  f1-score   support

          0       0.99      0.93      0.96      8807
          1       0.45      0.87      0.59       572

avg / total       0.96      0.93      0.94      9379



## LOGISTIC REGRESSION

In [174]:
from sklearn.linear_model import LogisticRegression

In [175]:
lr = LogisticRegression()

In [349]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [350]:
y_predi = lr.predict(X_test)

In [351]:
nn_accy = round(accuracy_score(y_predi, y_test), 3)
print (nn_accy,"\n")
nnconfmat = confusion_matrix(y_predi,y_test)
print(nnconfmat,"\n")
nnrep = classification_report(y_predi, y_test)
print(nnrep)

0.91 

[[8249  815]
 [  26  289]] 

             precision    recall  f1-score   support

          0       1.00      0.91      0.95      9064
          1       0.26      0.92      0.41       315

avg / total       0.97      0.91      0.93      9379



## SVM

In [ ]:
from sklearn.svm import SVC
svm=SVC(kernel='rbf')

In [ ]:
svm.fit(X_train, y_train)

In [354]:
y_predi=svm.predict(X_test)

In [355]:
nn_accy = round(accuracy_score(y_predi, y_test), 3)
print (nn_accy,"\n")
nnconfmat = confusion_matrix(y_predi,y_test)
print(nnconfmat,"\n")
nnrep = classification_report(y_predi, y_test)
print(nnrep)

0.884 

[[8265 1079]
 [  10   25]] 

             precision    recall  f1-score   support

          0       1.00      0.88      0.94      9344
          1       0.02      0.71      0.04        35

avg / total       1.00      0.88      0.93      9379

